In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

" 	1) Alcohol"	
" 	2) Malic acid"	
" 	3) Ash"	
"	4) Alcalinity of ash  "	
" 	5) Magnesium"	
"	6) Total phenols"	
" 	7) Flavanoids"	
" 	8) Nonflavanoid phenols"	
" 	9) Proanthocyanins"	
"	10)Color intensity"	
" 	11)Hue"	
" 	12)OD280/OD315 of diluted wines"	
" 	13)Proline            "	


######  Bayes Formula:  P(c/x)= P(x/x)*P(x) /  P(x)

###### P(c/x) = P(x1/c) * P(x2/c) * P(x3/c) * ....... * P(Xn/c)

###### P(c/x) is the posterior probability of class c given predictor ( features).
###### P(c) is the probability of class.
###### P(x/c) is the likelihood which is the probability of predictor given class.
###### P(x) is the prior probability of predictor.This term is ignored in this implementation since it doesn't affect which class distribution the sample is most likely to belong to.


In [2]:
df=pd.read_csv("wine.csv",names=["Wine","Alchol","Malic acid","Ash","Alcalinity of ash","Magnesium","Total Phenols",
                                 "Flavanoids","Nonflavanoid phenols","Proanthocyanins","Color intensity","Hue",
                                 "Diluted wined","Proline"])

In [3]:
df.head()

,Wine,Alchol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total Phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,Diluted wined,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [4]:
# we need to calculate probability of class

In [5]:
# Count of each class
df["Wine"].groupby(df["Wine"]).count()

Wine
1    59
2    71
3    48
Name: Wine, dtype: int64

###### Calculate prior probability

In [11]:
X=df.iloc[:,1:]
Y=df.iloc[:,0]

In [12]:
from sklearn.preprocessing import StandardScaler,LabelEncoder
sc=StandardScaler()
X=df.iloc[:,1:]
X=sc.fit_transform(np.array(X))
Y=df.iloc[:,0]
Y=np.array(Y)

In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.30)

###### Calculate prior probability

In [14]:
def caclulate_prior_proba(y):
    
    proba=np.mean(y_train==y)
     
    return proba 

In [15]:
caclulate_prior_proba(y=1)

0.3548387096774194

#### Caclculate mean and var of all the columns with respect to no.of classes 

In [16]:
# total no.of columns is 13
mean={}
var={}
for i,y in enumerate(np.unique(Y)):
    
    k=x_train[y_train==y].mean(axis=0)
    k1=x_train[y_train==y].var(axis=0)
    mean[i+1]=k
    var[i+1]=k1

###### likelihood
###### here we have total three classes which means wine types(1,2,3)

In [17]:
def likelihood(mean,var,x):
     
    # gausssian likelihood of x given mean and variance
    first= 1/np.sqrt(2*np.pi*np.power(var,2))
    second= np.exp(-np.power((x-mean),2)/(2*np.power(var,2)))
    gaussian=first*second
    
    return gaussian
    
    

In [18]:
likelihood(mean[1],var[1],x_train[0])

array([7.46525716e-08, 1.06658405e-01, 3.43571570e-01, 3.91741560e-09,
       3.07038346e-01, 2.73622374e-06, 9.16843159e-50, 5.42591393e-18,
       2.13856242e-02, 2.41046194e-02, 5.73830196e-09, 4.47368120e-18,
       2.08491029e-01])

In [19]:
classes=np.unique(Y)

In [48]:
def classify(samplex):
    
    posterior=[]
    
    for j in np.unique(Y):
        
        prior = caclulate_prior_proba(j)
        
        for sample,m,v in zip(samplex,mean[j],var[j]):
            
           
            
            like_hood=likelihood(m,v,sample)
            prior *= like_hood
    
        posterior.append(prior)  
    
    return classes[np.argmax(posterior)]


In [49]:
x_train[0]

array([-0.6801591 ,  0.62266555,  0.99967413,  2.25365574, -0.192495  ,
       -0.633101  , -1.45501851,  2.16066901, -0.79001277,  1.05629702,
       -1.26113805, -1.2453302 ,  0.42387759])

In [50]:
classify(x_train[0])

3

In [51]:
def predict(X):
        """ Predict the class labels of the samples in X """
        y_pred = [classify(sample) for sample in X]
        return y_pred

In [52]:
y_predict=np.array(predict(x_test))

In [53]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_predict)

array([[13,  2,  0],
       [ 0, 25,  0],
       [ 0,  3, 11]], dtype=int64)

###### accuracy score of test data

In [54]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_predict)

0.9074074074074074

###### accuracy score of train data

In [34]:
y_pred=np.array(predict(x_train))

In [36]:
confusion_matrix(y_train,y_pred)

array([[43,  1,  0],
       [ 0, 46,  0],
       [ 0,  4, 30]], dtype=int64)

In [35]:
accuracy_score(y_train,y_pred)

0.9596774193548387

###### Naive-Bayes from sklearn

In [526]:
from sklearn.preprocessing import StandardScaler,LabelEncoder
sc=StandardScaler()

In [17]:
X=df.iloc[:,1:]
X=sc.fit_transform(np.array(X))

In [18]:
Y=df.iloc[:,0]
Y=np.array(Y)

In [19]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.30)

In [40]:
from sklearn.naive_bayes import GaussianNB

In [41]:
model=GaussianNB()

In [42]:
model.fit(x_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [43]:
y_p=model.predict(x_test)

In [44]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_p)

0.9629629629629629

In [45]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_p)

array([[14,  1,  0],
       [ 0, 24,  1],
       [ 0,  0, 14]], dtype=int64)

In [46]:
y_p1=model.predict(x_train)

In [47]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train,y_p1)

0.9919354838709677

In [29]:
confusion_matrix(y_train,y_p1)

array([[40,  1,  0],
       [ 0, 51,  0],
       [ 0,  0, 32]], dtype=int64)